Google Colab Setup

In [ ]:
%pip install --upgrade \
    transformers \
    datasets \
    peft \
    accelerate \
    evaluate \
    fsspec \
    huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!rm -rf ~/.cache/huggingface/datasets
!rm -rf ~/.cache/huggingface/hub

LoRA finetune distilBert on GLUE-SST2 (`peft` version)

In [ ]:
import os
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, TaskType

# Load dataset (SST-2)
dataset = load_dataset("glue", "sst2")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples['sentence'], truncation=True)

dataset = dataset.map(preprocess_function, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

# Load base model and apply LoRA
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=[]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Define evaluation metric
metric = evaluate.load("glue", "sst2")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="no",
    save_strategy="no",
    report_to="none"
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

# Evaluate
eval_results = trainer.evaluate()
print("Eval results:", eval_results)

# Save model
if not os.path.exists("./model"):
    os.makedirs("./model")

save_path = os.path.join("./model", "lora_distilbert_sst2")
trainer.save_model(save_path)
print(f"[\u2713] Training complete. Model saved to {save_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


<ipython-input-3-54addb49eead>:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,0.478400
200,0.350100
300,0.301700
400,0.329300
500,0.339600
600,0.340500
700,0.288000
800,0.294400
900,0.333800
1000,0.292200


Eval results: {'eval_loss': 0.3038165271282196, 'eval_accuracy': 0.8956422018348624, 'eval_runtime': 1.3143, 'eval_samples_per_second': 663.452, 'eval_steps_per_second': 41.846, 'epoch': 3.0}
[✓] Training complete. Model saved to ./model/lora_distilbert_sst2


Predict

In [ ]:
from peft import PeftModel, PeftConfig

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

peft_config = PeftConfig.from_pretrained("./model/lora_distilbert_sst2")
base_model = AutoModelForSequenceClassification.from_pretrained(peft_config.base_model_name_or_path, num_labels=2)
model = PeftModel.from_pretrained(base_model, "./model/lora_distilbert_sst2")
model.eval()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=76

In [ ]:
import torch

sentence = "Having chicken for dinner is not bad."
inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
with torch.no_grad():
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()

label_map = {0: "Negative", 1: "Positive"}
print(f"Pred label: {label_map[prediction]}")

Pred label: Positive


Download

In [ ]:
!zip -r lora_distilbert_sst2 /content/model/lora_distilbert_sst2

  adding: content/model/lora_distilbert_sst2/ (stored 0%)
  adding: content/model/lora_distilbert_sst2/tokenizer.json (deflated 71%)
  adding: content/model/lora_distilbert_sst2/training_args.bin (deflated 52%)
  adding: content/model/lora_distilbert_sst2/adapter_model.safetensors (deflated 7%)
  adding: content/model/lora_distilbert_sst2/adapter_config.json (deflated 55%)
  adding: content/model/lora_distilbert_sst2/vocab.txt (deflated 53%)
  adding: content/model/lora_distilbert_sst2/tokenizer_config.json (deflated 75%)
  adding: content/model/lora_distilbert_sst2/special_tokens_map.json (deflated 42%)
  adding: content/model/lora_distilbert_sst2/README.md (deflated 66%)


In [ ]:
from google.colab import files
files.download('lora_distilbert_sst2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>